In [22]:
!nvidia-smi

Tue Sep 17 11:42:07 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [23]:
%%writefile exemplo1.cu
#include <thrust/device_vector.h> // Inclui a biblioteca para vetores na GPU (device)
#include <thrust/host_vector.h>   // Inclui a biblioteca para vetores na CPU (host)
#include <iostream>               // Biblioteca padrão para entrada e saída de dados

int main() {
    // Cria um vetor na CPU (host) com 5 elementos, todos inicializados com 0
    thrust::host_vector<double> host(5, 0);
    host[4] = 35; // Altera o último elemento do vetor para 35

    /* Os dados do vetor 'host' são copiados para a GPU,
       criando um vetor equivalente na GPU (device) */
    thrust::device_vector<double> dev(host);

    /* Altera o vetor na CPU, mas não afeta o vetor na GPU,
       pois a cópia já foi feita anteriormente */
    host[2] = 12; // Altera o terceiro elemento do vetor na CPU para 12

    // Exibe os elementos do vetor na CPU
    printf("Host vector: ");
    for (auto i = host.begin(); i != host.end(); i++) {
        std::cout << *i << " "; // Acesso rápido aos elementos na CPU
    }
    printf("\n");

    // Exibe os elementos do vetor na GPU
    printf("Device vector: ");
    for (auto i = dev.begin(); i != dev.end(); i++) {
        std::cout << *i << " "; // Acesso aos elementos na GPU é lento, pois os dados estão na GPU
    }
    printf("\n");
}

Overwriting exemplo1.cu


In [24]:
!nvcc -arch=sm_75 -std=c++14 exemplo1.cu -o exemplo1

In [25]:
!./exemplo1

Host vector: 0 0 12 0 35 
Device vector: 0 0 0 0 35 


In [26]:
%%writefile exemplo2.cu
#include <thrust/device_vector.h>              // Inclui a biblioteca para vetores na GPU
#include <thrust/host_vector.h>                // Inclui a biblioteca para vetores na CPU
#include <thrust/sequence.h>                   // Inclui a função para preencher vetores com uma sequência de números
#include <thrust/functional.h>                 // Inclui operações matemáticas padrão, como soma e multiplicação
#include <thrust/transform.h>                  // Inclui a função para transformar vetores com operações ponto a ponto
#include <thrust/iterator/constant_iterator.h> // Inclui o iterador constante necessário
#include <iostream>

int main() {
    // Cria um vetor na GPU (device_vector) com 10 elementos, todos inicializados com 0
    thrust::device_vector<double> V1(10, 0);
    // Preenche V1 com uma sequência de números: {0, 1, 2, ..., 9}
    thrust::sequence(V1.begin(), V1.end());

    // Cria um vetor na GPU com 5 elementos, todos inicializados com 0
    thrust::device_vector<double> V2(5, 0);
    // Preenche os dois primeiros elementos de V2 com 5.5: {5.5, 5.5, 0, 0, 0}
    thrust::fill(V2.begin(), V2.begin() + 2, 5.5);
    // Preenche os elementos restantes de V2 com 10: {5.5, 5.5, 10, 10, 10}
    thrust::fill(V2.begin() + 2, V2.end(), 10);

    // Cria dois vetores na GPU com 10 elementos, para armazenar resultados das operações
    thrust::device_vector<double> V3(10); // Vetor para armazenar o resultado da soma
    thrust::device_vector<double> V4(10); // Vetor para armazenar o resultado da multiplicação

    // Aplica a operação de soma elemento por elemento entre V1 e V2 e armazena o resultado em V3
    // Como V2 tem menos elementos, o restante de V1 é somado com zeros (elemento padrão).
    // Resultado: V3 = {0+5.5, 1+5.5, 2+10, 3+10, 4+10, 5+0, ..., 9+0}
    thrust::transform(V1.begin(), V1.end(), V2.begin(), V3.begin(), thrust::plus<double>());

    // Aplica multiplicação de V1 com o valor constante 0.5
    // e armazenaria o resultado em V4.
    thrust::transform(V1.begin(), V1.end(), thrust::constant_iterator<double>(0.5), V4.begin(), thrust::multiplies<double>());

    // Imprime os elementos de V1
    printf("V1: ");
    for (thrust::device_vector<double>::iterator i = V1.begin(); i != V1.end(); i++) {
        std::cout << *i << " "; // Acessa e imprime cada elemento do vetor V1
    }
    printf("\n");

    // Imprime os elementos de V2
    printf("V2: ");
    for (thrust::device_vector<double>::iterator i = V2.begin(); i != V2.end(); i++) {
        std::cout << *i << " "; // Acessa e imprime cada elemento do vetor V2
    }
    printf("\n");

    // Imprime os elementos de V3
    printf("V3: ");
    for (thrust::device_vector<double>::iterator i = V3.begin(); i != V3.end(); i++) {
        std::cout << *i << " "; // Acessa e imprime cada elemento do vetor V3
    }
    printf("\n");

    // Imprime o vetor V4, que foi criado mas não utilizado.
     printf("V4: ");
    for (thrust::device_vector<double>::iterator i = V4.begin(); i != V4.end(); i++) {
         std::cout << *i << " ";
    }
    printf("\n");

    return 0;
}

Overwriting exemplo2.cu


In [27]:
!nvcc -arch=sm_75 -std=c++14 exemplo2.cu -o exemplo2

In [28]:
!./exemplo2

V1: 0 1 2 3 4 5 6 7 8 9 
V2: 5.5 5.5 10 10 10 
V3: 5.5 6.5 12 13 14 5 6 7 8 9 
V4: 0 0.5 1 1.5 2 2.5 3 3.5 4 4.5 


In [29]:
%%writefile stocks.cu
#include <thrust/device_vector.h>   // Inclui a biblioteca Thrust para vetores na GPU
#include <thrust/host_vector.h>     // Inclui a biblioteca Thrust para vetores na CPU
#include <iostream>                 // Biblioteca padrão para entrada e saída de dados
#include <chrono>                   // Biblioteca para medir o tempo de execução do código
using namespace std;

int main() {
    int n = 2518;                   // Define o tamanho do vetor (2518 elementos)
    double value = 0.0;             // Variável auxiliar para armazenar o valor lido da entrada
    std::chrono::duration<double> diff; // Variável para armazenar a diferença de tempo calculada

    // Marca o início da medição do tempo de leitura dos dados
    auto leitura_i = std::chrono::steady_clock::now();

    // Cria um vetor na CPU (host_vector) com n elementos, todos inicializados com 0
    thrust::host_vector<double> host(n, 0);

    // Loop para ler n valores da entrada padrão e armazená-los no vetor host
    for (int i = 0; i < n; i++) {
        cin >> value;              // Lê um valor da entrada padrão
        host[i] = value;           // Armazena o valor lido no vetor host
    }

    // Marca o fim da medição do tempo de leitura dos dados
    auto leitura_f = std::chrono::steady_clock::now();

    // Calcula o tempo gasto na leitura dos dados
    diff = leitura_f - leitura_i;

    // Exibe o tempo de leitura em segundos
    cout << "Tempo de LEITURA (em segundos)  " << diff.count() << endl;

    // Marca o início da medição do tempo de cópia dos dados da CPU para a GPU
    auto copia_i = std::chrono::steady_clock::now();

    // Cria um vetor na GPU (device_vector) copiando os dados do vetor host da CPU
    thrust::device_vector<double> dev(host);

    // Marca o fim da medição do tempo de cópia dos dados
    auto copia_f = std::chrono::steady_clock::now();

    // Calcula o tempo gasto na cópia dos dados para a GPU
    diff = copia_f - copia_i;

    // Exibe o tempo de cópia em segundos
    cout << "Tempo de CÓPIA (em segundos)  " << diff.count() << endl;

    return 0;
}

Overwriting stocks.cu


In [30]:
!nvcc -arch=sm_70 -std=c++14 stocks.cu -o stocks

In [34]:
!./stocks < stocks-google.txt

Tempo de LEITURA (em segundos)  0.00203943
Tempo de CÓPIA (em segundos)  0.194415


Exercício 1

In [39]:
%%writefile ex1.cu
#include <thrust/device_vector.h>   // Inclui a biblioteca Thrust para vetores na GPU
#include <thrust/host_vector.h>     // Inclui a biblioteca Thrust para vetores na CPU
#include <iostream>                 // Biblioteca padrão para entrada e saída de dados
#include <chrono>                   // Biblioteca para medir o tempo de execução do código
using namespace std;

int main() {
    double value = 0.0;             // Variável auxiliar para armazenar o valor lido da entrada
    std::chrono::duration<double> diff; // Variável para armazenar a diferença de tempo calculada

    // Marca o início da medição do tempo de leitura dos dados
    auto leitura_i = std::chrono::steady_clock::now();

    // Cria um vetor na CPU (host_vector) sem tamanho inicial fixo
    thrust::host_vector<double> host;

    // Loop para ler valores da entrada padrão e armazená-los no vetor host
    while (cin >> value) {          // Continua lendo até que não haja mais dados de entrada
        host.push_back(value);      // Adiciona cada valor ao vetor host
    }

    // Marca o fim da medição do tempo de leitura dos dados
    auto leitura_f = std::chrono::steady_clock::now();

    // Calcula o tempo gasto na leitura dos dados
    diff = leitura_f - leitura_i;

    // Exibe o tempo de leitura em segundos
    cout << "Tempo de LEITURA (em segundos): " << diff.count() << endl;

    // Marca o início da medição do tempo de cópia dos dados da CPU para a GPU
    auto copia_i = std::chrono::steady_clock::now();

    // Cria um vetor na GPU (device_vector) copiando os dados do vetor host da CPU
    thrust::device_vector<double> dev(host);

    // Marca o fim da medição do tempo de cópia dos dados
    auto copia_f = std::chrono::steady_clock::now();

    // Calcula o tempo gasto na cópia dos dados para a GPU
    diff = copia_f - copia_i;

    // Exibe o tempo de cópia em segundos
    cout << "Tempo de CÓPIA (em segundos): " << diff.count() << endl;

    return 0;
}


Overwriting ex1.cu


In [49]:
!nvcc -arch=sm_70 -std=c++14 ex1.cu -o ex1

In [50]:
!./ex1 < stocks-google.txt

Tempo de LEITURA (em segundos): 0.00440269
Tempo de CÓPIA (em segundos): 0.200059


Exercício 2

In [53]:
%%writefile ex2.cu
#include <thrust/device_vector.h>   // Inclui a biblioteca Thrust para vetores na GPU
#include <thrust/host_vector.h>     // Inclui a biblioteca Thrust para vetores na CPU
#include <thrust/reduce.h>          // Inclui as funções de redução
#include <thrust/extrema.h>         // Inclui funções para encontrar máximos e mínimos
#include <iostream>                 // Biblioteca padrão para entrada e saída de dados
#include <chrono>                   // Biblioteca para medir o tempo de execução do código
using namespace std;

int main() {
    double value = 0.0;             // Variável auxiliar para armazenar o valor lido da entrada
    std::chrono::duration<double> diff; // Variável para armazenar a diferença de tempo calculada

    // Marca o início da medição do tempo de leitura dos dados
    auto leitura_i = std::chrono::steady_clock::now();

    // Cria um vetor na CPU (host_vector) sem tamanho inicial fixo
    thrust::host_vector<double> host;

    // Loop para ler valores da entrada padrão e armazená-los no vetor host
    while (cin >> value) {          // Continua lendo até que não haja mais dados de entrada
        host.push_back(value);      // Adiciona cada valor ao vetor host
    }

    // Marca o fim da medição do tempo de leitura dos dados
    auto leitura_f = std::chrono::steady_clock::now();

    // Calcula o tempo gasto na leitura dos dados
    diff = leitura_f - leitura_i;

    // Exibe o tempo de leitura em segundos
    cout << "Tempo de LEITURA (em segundos): " << diff.count() << endl;

    // Marca o início da medição do tempo de cópia dos dados da CPU para a GPU
    auto copia_i = std::chrono::steady_clock::now();

    // Cria um vetor na GPU (device_vector) copiando os dados do vetor host da CPU
    thrust::device_vector<double> dev(host);

    // Marca o fim da medição do tempo de cópia dos dados
    auto copia_f = std::chrono::steady_clock::now();

    // Calcula o tempo gasto na cópia dos dados para a GPU
    diff = copia_f - copia_i;

    // Exibe o tempo de cópia em segundos
    cout << "Tempo de CÓPIA (em segundos): " << diff.count() << endl;

    // --- Cálculo de Médias e Extremos ---

    // Cálculo da média dos preços
    double sum = thrust::reduce(dev.begin(), dev.end(), 0.0); // Soma dos preços
    double avg = sum / dev.size();                            // Média dos preços
    cout << "Preço médio das ações do Google: " << avg << endl;

    // Encontrar o preço mínimo e máximo
    auto minmax = thrust::minmax_element(dev.begin(), dev.end());
    cout << "Menor preço: " << *minmax.first << endl;
    cout << "Maior preço: " << *minmax.second << endl;

    // --- Cálculo para o último ano (365 dias) ---

    // Caso o vetor tenha mais de 365 elementos, calcula a média do último ano
    if (dev.size() >= 365) {
        // Cálculo da média dos últimos 365 dias
        double sum_last_year = thrust::reduce(dev.end() - 365, dev.end(), 0.0); // Soma dos últimos 365 dias
        double avg_last_year = sum_last_year / 365;                             // Média dos últimos 365 dias
        cout << "Preço médio nos últimos 365 dias: " << avg_last_year << endl;

        // Encontrar o preço mínimo e máximo nos últimos 365 dias
        auto minmax_last_year = thrust::minmax_element(dev.end() - 365, dev.end());
        cout << "Menor preço nos últimos 365 dias: " << *minmax_last_year.first << endl;
        cout << "Maior preço nos últimos 365 dias: " << *minmax_last_year.second << endl;
    } else {
        cout << "Menos de 365 dias de dados, não é possível calcular o último ano." << endl;
    }

    return 0;
}


Overwriting ex2.cu


In [54]:
!nvcc -arch=sm_70 -std=c++14 ex2.cu -o ex2

In [55]:
!./ex2 < stocks-google.txt

Tempo de LEITURA (em segundos): 0.00695575
Tempo de CÓPIA (em segundos): 0.204272
Preço médio das ações do Google: 1580.08
Menor preço: 1015.24
Maior preço: 2200
Preço médio nos últimos 365 dias: 1615.81
Menor preço nos últimos 365 dias: 1060.5
Maior preço nos últimos 365 dias: 2200


Exercício 3

In [61]:
%%writefile ex3.cu
#include <thrust/device_vector.h>   // Inclui a biblioteca Thrust para vetores na GPU
#include <thrust/host_vector.h>     // Inclui a biblioteca Thrust para vetores na CPU
#include <thrust/transform.h>       // Inclui a função para transformações ponto a ponto
#include <thrust/reduce.h>          // Inclui a função para operações de redução
#include <thrust/functional.h>      // Inclui operadores como minus
#include <iostream>                 // Biblioteca padrão para entrada e saída de dados
#include <fstream>                  // Para ler o arquivo de entrada
#include <sstream>                  // Para processar cada linha do arquivo
#include <string>                   // Manipulação de strings

int main() {
    // Vetores para armazenar os preços da Apple e Microsoft na CPU
    thrust::host_vector<double> AAPL_prices;
    thrust::host_vector<double> MSFT_prices;

    // Abrir o arquivo stocks2.txt
    std::ifstream file("stocks2.txt");
    std::string line;

    // Variáveis para armazenar os preços lidos de cada linha
    double aapl_price, msft_price;

    // Loop para ler os preços do arquivo
    while (std::getline(file, line)) {
        std::stringstream ss(line);    // Usa stringstream para dividir os valores
        char separator;                // Para ler a vírgula entre os valores

        // Ler o preço da Apple (AAPL)
        ss >> aapl_price;
        AAPL_prices.push_back(aapl_price);

        // Ignorar a vírgula e ler o preço da Microsoft (MSFT)
        ss >> separator >> msft_price;
        MSFT_prices.push_back(msft_price);
    }

    // Transferir os dados para a GPU
    thrust::device_vector<double> d_AAPL_prices = AAPL_prices;
    thrust::device_vector<double> d_MSFT_prices = MSFT_prices;

    // Vetor para armazenar as diferenças na GPU
    thrust::device_vector<double> differences(AAPL_prices.size());

    // Calcular a diferença ponto a ponto entre os dois vetores
    thrust::transform(d_AAPL_prices.begin(), d_AAPL_prices.end(), d_MSFT_prices.begin(), differences.begin(), thrust::minus<double>());

    // Calcular a soma das diferenças
    double total_difference = thrust::reduce(differences.begin(), differences.end(), 0.0);

    // Calcular a média das diferenças
    double avg_difference = total_difference / differences.size();

    if (avg_difference < 0) {
        std::cout << "A Apple teve valores maiores em suas ações." << std::endl;
    }

    // Exibir o resultado
    std::cout << "Diferença média entre os preços da Apple e Microsoft: " << avg_difference << std::endl;

    return 0;
}


Overwriting ex3.cu


In [62]:
!nvcc -arch=sm_70 -std=c++14 ex3.cu -o ex3

In [63]:
!./ex3 < stocks2.txt

A Apple teve valores maiores em suas ações.
Diferença média entre os preços da Apple e Microsoft: -265.232
